# Logisitc Regression Model using SK-Learn

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from numpy.random import randn
from numpy.matlib import repmat

from scipy.stats import norm
from scipy.optimize import fmin
from scipy.special import erf

from patsy import dmatrices

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer

In [2]:
# Utility for displaying our DataFrames
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

### First let's import the CV exercise to train on...

In [3]:
trainX = pd.read_csv('../match-data/trainX.csv')
trainy = pd.read_csv('../match-data/trainy.csv')

clean = pd.read_csv('../match-data/clean_all.csv')

display_side_by_side(trainy, trainX)

Al - Rawi Company for the manufacture of wires and cables Jordan 
 
 
 
 
 0 
 0 
 
 
 1 
 0 
 
 
 2 
 1 
 
 
 3 
 1 
 
 
 4 
 1 
 
 
 5 
 1 
 
 
 6 
 0 
 
 
 7 
 0 
 
 
 8 
 1 
 
 
 9 
 0 
 
 
 10 
 0 
 
 
 11 
 1 
 
 
 12 
 0 
 
 
 13 
 0 
 
 
 14 
 1 
 
 
 15 
 1 
 
 
 16 
 1 
 
 
 17 
 1 
 
 
 18 
 1 
 
 
 19 
 1 
 
 
 20 
 0 
 
 
 21 
 1 
 
 
 22 
 1 
 
 
 23 
 0 
 
 
 24 
 1 
 
 
 25 
 1 
 
 
 26 
 1 
 
 
 27 
 1 
 
 
 28 
 1 
 
 
 29 
 1 
 
 
 
 
 
 
 nationality 
 gender 
 age 
 education 
 night_shift 
 
 
 
 
 0 
 syrian 
 female 
 48 
 secondary 
 0 
 
 
 1 
 syrian 
 female 
 40 
 diploma 
 0 
 
 
 2 
 syrian 
 female 
 25 
 bachelors 
 0 
 
 
 3 
 syrian 
 male 
 35 
 secondary 
 0 
 
 
 4 
 syrian 
 male 
 38 
 diploma 
 1 
 
 
 5 
 syrian 
 female 
 21 
 bachelors 
 1 
 
 
 6 
 syrian 
 female 
 44 
 diploma 
 0 
 
 
 7 
 syrian 
 female 
 53 
 none 
 0 
 
 
 8 
 syrian 
 female 
 20 
 secondary 
 0 
 
 
 9 
 syrian 
 male 
 44 
 primary 
 1 
 
 
 10 
 syrian 
 male 
 51 
 primary 
 0 
 
 
 11 
 syrian 
 male 
 34 
 secondary 
 0 
 
 
 12 
 syrian 
 female 
 41 
 secondary 
 0 
 
 
 13 
 syrian 
 female 
 40 
 primary 
 0 
 
 
 14 
 syrian 
 female 
 37 
 diploma 
 0 
 
 
 15 
 syrian 
 male 
 26 
 secondary 
 1 
 
 
 16 
 syrian 
 male 
 38 
 diploma 
 1 
 
 
 17 
 syrian 
 female 
 32 
 bachelors 
 0 
 
 
 18 
 syrian 
 male 
 27 
 primary 
 1 
 
 
 19 
 syrian 
 male 
 29 
 secondary 
 0 
 
 
 20 
 syrian 
 male 
 45 
 primary 
 1 
 
 
 21 
 syrian 
 male 
 20 
 secondary 
 1 
 
 
 22 
 syrian 
 female 
 28 
 secondary 
 0 
 
 
 23 
 syrian 
 female 
 52 
 secondary 
 0 
 
 
 24 
 syrian 
 male 
 32 
 bachelors 
 1 
 
 
 25 
 syrian 
 male 
 24 
 bachelors 
 1 
 
 
 26 
 syrian 
 female 
 36 
 secondary 
 0 
 
 
 27 
 syrian 
 male 
 22 
 secondary 
 1 
 
 
 28 
 syrian 
 female 
 22 
 secondary 
 1 
 
 
 29 
 syrian 
 female 
 27 
 masters 
 1

Next up we create some utilities that we will use later to one-hot encode our categorical features such as nationality, gender, etc.

In [4]:
def array_vector(col):
    return np.array(str(col))

arrayerize = np.vectorize(array_vector)

def one_hot_encode(df, column):
    # This is gross but since strings are iterable, we have to wrap them in a list
    # in order for the binarizer to parse the labels as strings and not chars
    labels = arrayerize(pd.DataFrame(df[column]))
    terms = arrayerize(pd.DataFrame(list(set(df[column]))))

    mlb = MultiLabelBinarizer()
    mlb.fit(terms)
    mlb.transform(labels)
    columns = [ f'{column}-{classname}' for classname in  mlb.classes_]

    encoded = pd.DataFrame(mlb.transform(labels), columns=columns, index=df[column].index)

    df.drop(column, axis=1, inplace=True)
    return df.join(encoded)

### Pre Process our training data

In [5]:
trainX = one_hot_encode(trainX, 'education')
trainX = one_hot_encode(trainX, 'gender')
trainX = one_hot_encode(trainX, 'nationality')
trainX['nationality-jordanian'] = 0
trainX['nationality-unknown'] = 0
trainX['education-college'] = 0
trainX['education-other'] = 0

In [6]:
trainX

age  night_shift  education-bachelors  education-diploma  \
0    48            0                    0                  0   
1    40            0                    0                  1   
2    25            0                    1                  0   
3    35            0                    0                  0   
4    38            1                    0                  1   
5    21            1                    1                  0   
6    44            0                    0                  1   
7    53            0                    0                  0   
8    20            0                    0                  0   
9    44            1                    0                  0   
10   51            0                    0                  0   
11   34            0                    0                  0   
12   41            0                    0                  0   
13   40            0                    0                  0   
14   37            0                    0                  1   
15   26            1                    0                  0   
16   38            1                    0                  1   
17   32            0                    1                  0   
18   27            1                    0                  0   
19   29            0                    0                  0   
20   45            1                    0                  0   
21   20            1                    0                  0   
22   28            0                    0                  0   
23   52            0                    0                  0   
24   32            1                    1                  0   
25   24            1                    1                  0   
26   36            0                    0                  0   
27   22            1                    0                  0   
28   22            1                    0                  0   
29   27            1                    0                  0   

    education-masters  education-none  education-primary  education-secondary  \
0                   0               0                  0                    1   
1                   0               0                  0                    0   
2                   0               0                  0                    0   
3                   0               0                  0                    1   
4                   0               0                  0                    0   
5                   0               0                  0                    0   
6                   0               0                  0                    0   
7                   0               1                  0                    0   
8                   0               0                  0                    1   
9                   0               0                  1                    0   
10                  0               0                  1                    0   
11                  0               0                  0                    1   
12                  0               0                  0                    1   
13                  0               0                  1                    0   
14                  0               0                  0                    0   
15                  0               0                  0                    1   
16                  0               0                  0                    0   
17                  0               0                  0                    0   
18                  0               0                  1                    0   
19                  0               0                  0                    1   
20                  0               0                  1                    0   
21                  0               0                  0                    1   
22                  0               0                  0                    1   
23                  0               0                  0                   

In [7]:
X = trainX
y = trainy.T.values[0]

model2 = LogisticRegression(max_iter=1000)
model2.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Pre Process our training data

In [8]:
clean = clean.drop(['Unnamed: 0', 'lat', 'lng'], axis=1)
clean = clean[clean['age'] != 0]
clean.dropna(inplace=True)

testX = clean.copy(deep=True)
testX = one_hot_encode(testX, 'education')
testX = one_hot_encode(testX, 'gender')
testX = one_hot_encode(testX, 'nationality')

In [9]:
testX

age  night-shift  education-bachelors  education-college  \
0     48          0.0                    0                  0   
2     44          1.0                    0                  0   
3     28          0.0                    0                  0   
4     40          0.0                    0                  0   
6     53          0.0                    0                  0   
7     20          0.0                    0                  0   
8     19          0.0                    0                  0   
9     25          0.0                    1                  0   
10    45          1.0                    0                  0   
11    18          0.0                    0                  0   
12    44          0.0                    0                  0   
15    27          1.0                    0                  0   
16    43          0.0                    0                  0   
17    38          0.0                    0                  0   
18    42          1.0                    0                  0   
19    33          1.0                    0                  0   
20    34          0.0                    0                  0   
21    24          1.0                    1                  0   
22    45          0.0                    0                  0   
23    27          1.0                    0                  0   
24    44          0.0                    0                  0   
25    41          1.0                    0                  0   
26    36          0.0                    0                  0   
27    26          0.0                    0                  0   
28    22          1.0                    0                  0   
29    59          1.0                    1                  0   
30    32          0.0                    0                  0   
31    25          1.0                    1                  0   
32    35          1.0                    0                  0   
33    47          0.0                    0                  0   
..   ...          ...                  ...                ...   
395   49          1.0                    0                  0   
396   47          0.0                    1                  0   
397   45          0.0                    0                  0   
398   42          0.0                    0                  0   
399   20          0.0                    0                  0   
400   29          0.0                    0                  0   
401   29          0.0                    0                  0   
402   42          0.0                    0                  0   
403   48          0.0                    0                  0   
404   36          1.0                    0                  0   
405   46          0.0                    0                  0   
406   43          0.0                    0                  1   
407   57          0.0                    0                  0   
408   34          1.0                    0                  0   
409   48          0.0                    0                  0   
410   44          0.0                    0                  0   
411   35          0.0                    0                  0   
412   24          0.0                    0                  0   
413   47          1.0                    0                  0   
414   48          0.0                    0                  0   
415   22          1.0                    0                  1   
416   50          1.0                    0                  0   
417   46          1.0                    0                  0   
418   40          0.0                    0                  0   
419   27          1.0                    0                  0   
420   18          1.0                    0                  0   
421   28          0.0                    0                  1   
422   33          0.0                    0                  0   
423   34          0.0                    0                  0   
424   34          0.0                   

### And then lets take our set of real candidates and predict their probabilities

In [10]:

predicted = model2.predict(testX)
probs = model2.predict_proba(testX)

### Comparison of our predictions vs the true generated probabilities

In [11]:
display_side_by_side(pd.DataFrame(probs.T[1], columns=['candidate fit']), 
                     pd.DataFrame(clean))

candidate fit 
 
 
 
 
 0 
 0.297379 
 
 
 1 
 0.167812 
 
 
 2 
 0.710096 
 
 
 3 
 0.295151 
 
 
 4 
 0.048137 
 
 
 5 
 0.831755 
 
 
 6 
 0.843684 
 
 
 7 
 0.736218 
 
 
 8 
 0.258193 
 
 
 9 
 0.854915 
 
 
 10 
 0.375503 
 
 
 11 
 0.751361 
 
 
 12 
 0.396303 
 
 
 13 
 0.350783 
 
 
 14 
 0.604142 
 
 
 15 
 0.346242 
 
 
 16 
 0.591254 
 
 
 17 
 0.658196 
 
 
 18 
 0.355154 
 
 
 19 
 0.628250 
 
 
 20 
 0.227650 
 
 
 21 
 0.207861 
 
 
 22 
 0.373001 
 
 
 23 
 0.607734 
 
 
 24 
 0.723848 
 
 
 25 
 0.081875 
 
 
 26 
 0.213545 
 
 
 27 
 0.638182 
 
 
 28 
 0.307642 
 
 
 29 
 0.067835 
 
 
 30 
 0.051273 
 
 
 31 
 0.499504 
 
 
 32 
 0.450986 
 
 
 33 
 0.843684 
 
 
 34 
 0.776777 
 
 
 35 
 0.718821 
 
 
 36 
 0.687415 
 
 
 37 
 0.513119 
 
 
 38 
 0.521653 
 
 
 39 
 0.460695 
 
 
 40 
 0.193778 
 
 
 41 
 0.067835 
 
 
 42 
 0.594676 
 
 
 43 
 0.209256 
 
 
 44 
 0.334210 
 
 
 45 
 0.499046 
 
 
 46 
 0.515192 
 
 
 47 
 0.136513 
 
 
 48 
 0.289266 
 
 
 49 
 0.170624 
 
 
 50 
 0.164229 
 
 
 51 
 0.652641 
 
 
 52 
 0.226016 
 
 
 53 
 0.543287 
 
 
 54 
 0.340407 
 
 
 55 
 0.128066 
 
 
 56 
 0.724021 
 
 
 57 
 0.658196 
 
 
 58 
 0.113320 
 
 
 59 
 0.417484 
 
 
 60 
 0.459463 
 
 
 61 
 0.331063 
 
 
 62 
 0.494720 
 
 
 63 
 0.180431 
 
 
 64 
 0.201948 
 
 
 65 
 0.479899 
 
 
 66 
 0.791628 
 
 
 67 
 0.129916 
 
 
 68 
 0.226168 
 
 
 69 
 0.438973 
 
 
 70 
 0.258359 
 
 
 71 
 0.048727 
 
 
 72 
 0.311919 
 
 
 73 
 0.521653 
 
 
 74 
 0.460695 
 
 
 75 
 0.474038 
 
 
 76 
 0.258193 
 
 
 77 
 0.237713 
 
 
 78 
 0.698375 
 
 
 79 
 0.222686 
 
 
 80 
 0.275360 
 
 
 81 
 0.141528 
 
 
 82 
 0.723848 
 
 
 83 
 0.071856 
 
 
 84 
 0.632909 
 
 
 85 
 0.183278 
 
 
 86 
 0.455729 
 
 
 87 
 0.663994 
 
 
 88 
 0.314969 
 
 
 89 
 0.591254 
 
 
 90 
 0.854915 
 
 
 91 
 0.371027 
 
 
 92 
 0.682016 
 
 
 93 
 0.199173 
 
 
 94 
 0.689414 
 
 
 95 
 0.843684 
 
 
 96 
 0.705961 
 
 
 97 
 0.164229 
 
 
 98 
 0.744866 
 
 
 99 
 0.309413 
 
 
 100 
 0.648510 
 
 
 101 
 0.499721 
 
 
 102 
 0.705961 
 
 
 103 
 0.325357 
 
 
 104 
 0.628452 
 
 
 105 
 0.881096 
 
 
 106 
 0.698375 
 
 
 107 
 0.788309 
 
 
 108 
 0.575453 
 
 
 109 
 0.532098 
 
 
 110 
 0.254545 
 
 
 111 
 0.577587 
 
 
 112 
 0.481328 
 
 
 113 
 0.057541 
 
 
 114 
 0.199173 
 
 
 115 
 0.090835 
 
 
 116 
 0.044816 
 
 
 117 
 0.658196 
 
 
 118 
 0.437678 
 
 
 119 
 0.727299 
 
 
 120 
 0.494720 
 
 
 121 
 0.307642 
 
 
 122 
 0.213545 
 
 
 123 
 0.383620 
 
 
 124 
 0.725707 
 
 
 125 
 0.412837 
 
 
 126 
 0.293396 
 
 
 127 
 0.723848 
 
 
 128 
 0.183278 
 
 
 129 
 0.118580 
 
 
 130 
 0.319113 
 
 
 131 
 0.644136 
 
 
 132 
 0.477573 
 
 
 133 
 0.741046 
 
 
 134 
 0.232051 
 
 
 135 
 0.258359 
 
 
 136 
 0.093698 
 
 
 137 
 0.703010 
 
 
 138 
 0.271558 
 
 
 139 
 0.209256 
 
 
 140 
 0.391733 
 
 
 141 
 0.073601 
 
 
 142 
 0.642788 
 
 
 143 
 0.526426 
 
 
 144 
 0.474871 
 
 
 145 
 0.226016 
 
 
 146 
 0.518770 
 
 
 147 
 0.453080 
 
 
 148 
 0.464219 
 
 
 149 
 0.706141 
 
 
 150 
 0.180431 
 
 
 151 
 0.207861 
 
 
 152 
 0.516660 
 
 
 153 
 0.788309 
 
 
 154 
 0.913250 
 
 
 155 
 0.186291 
 
 
 156 
 0.460695 
 
 
 157 
 0.843684 
 
 
 158 
 0.724021 
 
 
 159 
 0.371027 
 
 
 160 
 0.239787 
 
 
 161 
 0.226168 
 
 
 162 
 0.798595 
 
 
 163 
 0.136411 
 
 
 164 
 0.214379 
 
 
 165 
 0.615647 
 
 
 166 
 0.264794 
 
 
 167 
 0.373001 
 
 
 168 
 0.211030 
 
 
 169 
 0.147086 
 
 
 170 
 0.831755 
 
 
 171 
 0.081232 
 
 
 172 
 0.741046 
 
 
 173 
 0.352710 
 
 
 174 
 0.258359 
 
 
 175 
 0.776777 
 
 
 176 
 0.653056 
 
 
 177 
 0.744866 
 
 
 178 
 0.710096 
 
 
 179 
 0.262043 
 
 
 180 
 0.526426 
 
 
 181 
 0.691698 
 
 
 182 
 0.612287 
 
 
 183 
 0.144701 
 
 
 184 
 0.412627 
 
 
 185 
 0.296344 
 
 
 186 
 0.407282 
 
 
 187 
 0.307642 
 
 
 188 
 0.180431 
 
 
 189 
 0.228658 
 
 
 190 
 0.103180 
 
 
 191 
 0.831755 
 
 
 192 
 0.736218 
 
 
 193 
 0.736218 
 
 
 194 
 0.1706